In [ ]:
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.tools import toolbox

#### Fingerprints

In [ ]:
#####
# DD analysis from single file (multiple msmts allowed)
####
from analysis.scripts.Quantum_Memory import Simple_Decoupling_Analysis as sda
reload(sda)

name_contains = '128pulses_9'

ssro_calib_folder = toolbox.latest_data(contains='AdwinSSRO_SSROCalibration')
timestamp = None#'20150605163851'
nr_ids =60

# '20150602022709'] = [11, r'D:\measuring\data\20150530\235230_AdwinSSRO_SSROCalibration_Gretel_sil2']

if timestamp != None:
    if nr_ids > 1:
        x, y, yerr, folder, all_stitching_points = sda.get_data_multiple_msmts(name_contains, nr_ids, input_timestamp = timestamp, ssro_calib_folder = ssro_calib_folder)
    else:
        x, y, yerr, folder = sda.get_data(name_contains, input_timestamp = timestamp, ssro_calib_folder = ssro_calib_folder)
else:
#     x, y, yerr, folder, timestamp = sda.get_data_multiple_msmts(name_contains, nr_ids)
    if nr_ids > 1:
        x, y, yerr, folder, timestamp, all_stitching_points = sda.get_data_multiple_msmts(name_contains, nr_ids, input_timestamp = timestamp, ssro_calib_folder = ssro_calib_folder)
    else:
        x, y, yerr, folder, timestamp = sda.get_data(name_contains, ssro_calib_folder = ssro_calib_folder)
print folder
f = plt.figure(figsize=(25,6))
ax = f.add_subplot(1,1,1)

plt.errorbar(x,y,yerr = yerr, fmt = '-o',color='RoyalBlue')
plt.ylabel(r'Signal corrected for RO',fontsize=25)
plt.xlabel('tau (us)',fontsize=25)
# plt.xlabel('Free evolution time (us)',fontsize=25)
plt.tick_params(axis='x', labelsize=25)
plt.tick_params(axis='y', labelsize=25)
plt.title(timestamp)
plt.savefig(os.path.join(folder, 'SimpleDecoupling.png'))

# Add stitching points as vertical lines (if they exist)
ymin, ymax = ax.get_ylim()
ax.set_ylim([0,1.05])
ax.set_xlim([50.5,52.5])

if len(all_stitching_points) != 0:
    #plt.vlines(all_stitching_points, ymin, 1, linestyles = 'dashed', label = 'stitching_points')
    lines=0
    for val in all_stitching_points:
        #plt.text(val, 1., str(val))
        lines=0

plt.savefig(os.path.join(folder, 'SimpleDecoupling_stitching_pts.png'))

all_stitching_points = []

#### Simulate fingerprint

In [ ]:
import numpy as np
from analysis.lib.tools import toolbox
from analysis.lib.m2.ssro import mbi
import analysis.lib.QEC.nuclear_spin_characterisation as SC #used for simulating FP response of spins
import matplotlib.cm as cm
from matplotlib import pyplot as plt
import analysis.lib.QEC.hyperfine_params as hyperfine_params; reload(hyperfine_params) 

hyperfine_params = {}
hyperfine_params['C1']  = {'par' : 0.25e3, 'perp':.18e3}
hyperfine_params['C2']  = {'par' : -0.65e3, 'perp':0.22e3}
hyperfine_params['C3']  = {'par' : 0.02e3, 'perp':.02e3}
hyperfine_params['C4']  = {'par' : 0.1e3, 'perp':0.1e3}
hyperfine_params['C5']  = {'par' : 0.07e3, 'perp':0.04e3}
hyperfine_params['C6']  = {'par' : 0.5e3, 'perp':.01e3}
hyperfine_params['C7']  = {'par' : 0.05e3, 'perp':0.008e3}
hyperfine_params['C8']  = {'par' : 0.25e3, 'perp':0.01e3}
hyperfine_params['C9']  = {'par' : 0.32e3, 'perp':0.05e3}

hf = hyperfine_params

ZFS                 = 2.87180e9
g_factor            = 2.8025e6

def plot_sim_vs_Bx(spin_list=['C1'],Bx_list = [0],B_Field = 12, N =2):

    for ii in range(len(spin_list)):
        print spin_list
        fig = figure(ii+1,figsize=(25,6))
        ax = fig.add_subplot(111)
        # ax.title('Vary Bx for Spin '+ spin_list[ii])
        start, end = ax.get_xlim()
        # ax.xaxis.set_ticks(np.arange(start, end, 0.1))

        for b in range(len(Bx_list)):
            Bx = Bx_list[b]
            #print spin_list[ii]

            HF_par=[hf[spin_list[ii]]['par']  - hf[spin_list[ii]]['perp']*Bx/B_Field]#HF_par 	= [10e3]
            HF_perp=[hf[spin_list[ii]]['perp'] + hf[spin_list[ii]]['par']*Bx/B_Field]#HF_perp = [100e3]
            '''
            print Bx/B_Field

            print hf[spin_list[ii]]['par']
            print hf[spin_list[ii]]['perp']

            print HF_par
            print HF_perp
            '''
            print 'ii =' ,ii
            
            tau_lst = np.linspace(30e-6,32e-6,960)
            if ii == 0:
                Mt = SC.dyn_dec_signal(HF_par,HF_perp,B_Field,N,tau_lst)
            else:
                Mt=Mt*SC.dyn_dec_signal(HF_par,HF_perp,B_Field,N,tau_lst)
            #if ii == 0:
            FP_signal = ((Mt+1)/2)
            #else:
            #    FP_signal = FP_signal*((Mt+1)/2)
            ax.plot(tau_lst*1e6, FP_signal[0,:],'.-',lw=.8,label = 'spin_'+spin_list[ii]+'_Bx_'+str(Bx))
    ax.set_xlabel('tau (us)')
    ax.set_ylim([0.0,1.05])
    plt.legend(loc=4)

# plot_sim_vs_Bx(spin_list=['C1'],Bx_list = [8.5],B_Field = 22, N =8)
# plot_sim_vs_Bx(spin_list=['C1'],Bx_list = [8],B_Field = 22, N =16)
plot_sim_vs_Bx(spin_list=['C1','C2'],Bx_list = [2.7],B_Field = 22.6, N =400)
# plot_sim_vs_Bx(spin_list=['C1'],Bx_list = [8],B_Field = 22, N =64)
# plot_sim_vs_Bx(spin_list=['C1'],Bx_list = [7],B_Field = 22, N =8)
# ['C1','C3','C10']   ,'C2','C3','C4','C5','C6','C7']

#### Carbon rabi (DD sweep N)

In [ ]:
#### Carbon control sweep N analysis #####
from analysis.scripts.QEC import Carbon_control_analysis_sweep_N as ccan
reload(ccan)


def plot_single_Crabi(timestamp=None,contains=None,do_print=False):
    guess_frq = [1./500]
    guess_amp = [0.5]
    guess_decay = [1000]
    guess_phase = [180]
    guess_offset = 0.5
    show_guess = False
    plot_fit = True
    fixed = [3] # freq, offset, amp, phase, decay
    fit_result = ccan.Carbon_control_sweep_N(timestamp=timestamp,contains=contains,frequency = guess_frq,amplitude = guess_amp,decay_constant = guess_decay,
            phase = guess_phase,offset = guess_offset, plot_fit = plot_fit,show_guess = show_guess,do_print = do_print,fixed = fixed)
    if fit_result!=None:
        return fit_result
amplitudes=[]
frequencies=[]
tau=[]
nr_of_points=1
for i in np.arange(6):
    contains_str='N_C1_tauidx_'+str(i)
    #timestamp = '20150715160714' Note: overwritten below and by contains
    timestamp=None
    f_r=plot_single_Crabi(contains=contains_str)    
    amplitudes.append(f_r['params'][2])
    frequencies.append(f_r['params'][3])
    tau.append(f_r['tau']*1e6)

fig=plt.figure()
ax=plt.subplot(211)
ax.plot(tau,amplitudes,'o',color='Crimson')
ax.set_ylabel('amplitude')
ax.set_xticks([])
ax2=plt.subplot(212)
ax2.plot(tau,frequencies,'o',color='Royalblue')
ax2.set_ylabel('frequency')
ax2.set_xlabel('tau [us]')


In [ ]:
name='_C1pi_750ns'
toolbox.latest_data(contains='positive_Tomo_Z1'+name)

### Carbon ramsey (Ren - MBI - wait - Z-Gate - Ren )

In [ ]:
from analysis.scripts.QEC import carbon_ramsey_analysis as cr 
reload(cr)
c=1
f0, uf0 = cr.Carbon_Ramsey(timestamp=None, 
              offset = 0.5, amplitude = 0.4, x0=0, decay_constant =.3, exponent = 2, 
              frequency = 0.2e3, phase =-90, 
              plot_fit = False, show_guess = False,fixed = [2,3,4],            
              return_freq = True,
              return_results = False,
              title = '_msm1_freq_C'+str(c))




## Carbon ramsey (Ren - MBI - wait - Z-Gate - Ren )

In [ ]:
import analysis.scripts.QEC.carbon_ramsey_analysis as cramsey
reload(cramsey)
cramsey.Carbon_Ramsey_DD_freq(older_than=None,carbon=1,
                              frequency=24.598e3, phase=-120,amplitude=0.25,decay_constant=10,fixed=[2,3,4],
                              plot_fit=True, show_guess=False)

In [ ]:
###############################
###### Stitch ramseys##########
###############################

In [ ]:
from analysis.scripts.QEC import carbon_ramsey_analysis as cr 
reload(cr)
cr.Carbon_Ramsey_mult_msmts(timestamp=None, 
              offset = 0.5, amplitude = 0.4, x0=0, decay_constant = .3, exponent = 2, 
              frequency = 0.003e3, phase =-90, 
              plot_fit = True, show_guess = False,fixed = [2],            
              return_freq = True,
              return_results = False,
              title = '_msm0_freq_C'+str(c),
              contains=['120744_CarbonRamseyInitialised_Gretel_msm0_freq_C1','125055_CarbonRamseyInitialised_Gretel_msm0_freq_C1'])

## Optimize Carbon Gates (Sweep n,tau)

In [ ]:
from analysis.scripts.QEC import Carbon_gate_calibration_v2 as Cgatecal
reload(Cgatecal)

Cgatecal.get_gate_fidelity(carbon=1,older_than=None)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# some data
x = np.arange(0, 10, 0.1)
y1 = np.sin(x)
y2 = np.cos(x)

# plot of the data
fig = plt.figure()
ax = fig.add_axes([0.1, 0.1, 0.6, 0.75])
ax.plot(x, y1,'-k', lw=2, label='black sin(x)')
ax.plot(x, y2,'-r', lw=2, label='red cos(x)')
ax.set_xlabel('x', size=22)
ax.set_ylabel('y', size=22)
ax.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)

plt.show()

In [ ]:
plt?